In [1]:
from DS_detector import gen_pics_for_detection, detect_clusters
from DS_Planck_Unet import *
from DS_data_transformation import draw_df
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from tensorflow import convert_to_tensor
from DS_healpix_fragmentation import one_pixel_fragmentation, matr2dict, \
    draw_proper_circle, pix2radec
import os

In [2]:
model = load_planck_model(
    '/home/rt2122/Models/planck_z/f8d2.ep0040.hdf5')

In [3]:
thr_list = [0.1 * i for i in range(9, 0, -1)]

In [ ]:
for i in tqdm(range(48)):
    all_dict = gen_pics_for_detection(i, model, step=16)
    
    stats = {cat : [] for cat in all_dict['true_clusters']}
    for thr in tqdm(thr_list):
        cur_stats = detect_clusters(all_dict, thr, all_catalogs_mode=True, binary=False,  
                                    max_dist=5/60)
        for cat in cur_stats:
            stats[cat].append(pd.DataFrame(cur_stats[cat], index=[thr]))

    for cat in stats:
        if len(stats[cat]) > 1:
            stats[cat] = pd.concat(stats[cat])
        elif len(stats[cat]) > 0:
            stats[cat] = stats[cat][0]
        else:
            continue
        stats[cat].index.name = 'thr'
        stats[cat].to_csv(os.path.join('/home/rt2122/Data/scans/', cat+str(i) + '.csv'))
    

In [ ]:
draw_df({cat : stats[cat]['tp'] / (stats[cat]['tp'] + stats[cat]['fn']) 
             for cat in ['planck_z', 'planck_no_z', 'mcxcwp', 'act']},
        thr_list, ygrid=np.arange(0,1.1, 0.1), comment='ep 40', comm_coords=(0.6, 0.2))

In [ ]:
draw_df({'fp':stats['all']['fp']},
        thr_list, ygrid=np.arange(0,600,100), comment='ep 40', comm_coords=(0.6, 100))

In [ ]:
stats['act']

In [ ]:
model = load_planck_model(
    '/home/rt2122/Models/planck_z/f8d2.ep0014-vl0.006570-l0.004067.hdf5')

In [ ]:
all_dict = gen_pics_for_detection(2, model, step=16)

In [ ]:
stats = {cat : [] for cat in all_dict['true_clusters']}
for thr in tqdm(thr_list):
    cur_stats = detect_clusters(all_dict, thr, all_catalogs_mode=True, binary=False,  
                                max_dist=5/60)
    for cat in all_dict['true_clusters']:
        stats[cat].append(pd.DataFrame(cur_stats[cat], index=[thr]))
    
for cat in all_dict['true_clusters']:
    stats[cat] = pd.concat(stats[cat])

In [ ]:
draw_df({cat : stats[cat]['tp'] / (stats[cat]['tp'] + stats[cat]['fn']) 
             for cat in ['planck_z', 'planck_no_z', 'mcxcwp', 'act']},
        thr_list, ygrid=np.arange(0,1.1, 0.1), comment='ep 14', comm_coords=(0.6, 0.2))

In [ ]:
draw_df({'fp':stats['all']['fp']},
        thr_list, ygrid=np.arange(0,700,100), comment='ep 14', comm_coords=(0.6, 100))

In [ ]:
stats['act']